In [2]:
import os
import pyxdf
import matplotlib.pyplot as plt
import numpy as np
import mne
from mnelab.io import read_raw
import pynt
from sklearn.model_selection import cross_val_score
from scipy.io import loadmat
from scipy.signal import resample
import seaborn
seaborn.set_context("paper") 
from mne.bem import fit_sphere_to_headshape 
from scipy import interpolate
from scipy.interpolate import griddata

In [ ]:
path = "/Users/niluf/BCI_thesis/New_rc/BachelorThesis2324Workspace/s1038443ThesisData/P1Data"
repo = "/Users/niluf/BCI_thesis/New_rc"
subject = "sub-P001"
session = "ses-S001"
task = "cvep"  
fs = 240
l_freq = 1
h_freq = 35
trial_time = 4.2
         
b1 = "1"
b2 = "2"
b3 = "3"
b4 = "4"
b5 = "5"
b6 = "6"
b7 = "7" 

pr1= 40
pr2= 240
pr3= 80
pr4= 60
pr5= 30
pr6= 48
pr7= 120

def settingsProcessingData(pr, b, trial_time, h_freq, l_freq, fs, subject, session, task, repo, path):
       
    # Load xdf data into MNE
    print("Loading xdf data into MNE.")
    fn = os.path.join(path, subject, session, "eeg", f"{subject}_{session}_task-{task}_run-{b}_eeg.xdf")
    streams = pyxdf.resolve_streams(fn)
    names = [stream["name"] for stream in streams]
    stream_id = streams[names.index("BioSemi")]["stream_id"]
    raw = read_raw(fn, stream_ids=[stream_id])

    # Filtering
    print("Filtering.")
    raw = raw.filter(l_freq=l_freq, h_freq=h_freq, picks=np.arange(1, 65))

    # Read events
    # N.B. need to adjust marker channel data
    print("Reading events.")
    raw._data[0, :] -= np.min(raw._data[0, :])
    raw._data[0, raw._data[0, :] > 0] = 1
    events = mne.find_events(raw, stim_channel="Trig1")

    # Slicing
    print("Slicing.")
    baseline = 0.5
    epo = mne.Epochs(raw, events=events, tmin=-baseline, tmax=trial_time, baseline=None, picks="eeg", preload=True)

    # Resampling
    print("Resampling.")
    epo = epo.resample(sfreq=fs)

    # Extract data
    X = epo.get_data()[:, :, int(baseline * epo.info["sfreq"]):].transpose((1, 2, 0))  # channels(64) x samples(1008) x trials(32)
    print(f"The shape of X shape : {X.shape}" )

    keyboard_stream_index = 0
    biosemi_stream_index = 1

    # Load marker stream
    data, header = pyxdf.load_xdf(fn)
    marker_stream = data[keyboard_stream_index]

    # Extract labels
    labels = [int(marker[3]) for marker in marker_stream["time_series"] if marker[2] == "target"]
    print("Found", len(labels), "labels.")
    y = np.array(labels).astype("uint8")
    print(f"The shape of y shape : {y.shape}" )
    
    # Create output folder
    if not os.path.exists(os.path.join(path, "derivatives", subject)):
        os.makedirs(os.path.join(path, "derivatives", subject))

    # Load codes
    code = "mgold_61_6521"
    V = np.load(os.path.join(repo, "experiment", "codes", f"{code}.npz"))["codes"]
    V = np.repeat(V, int(fs / pr), axis=0).astype("uint8")
    print("V:", V.shape)

    # Save data
    np.savez(os.path.join(path, "derivatives", subject, f"{subject}_{b}.npz"),
        X=X, y=y, V=V, fs=fs) 


# ----------------------- Processing of the data --------------------   

# Participant - block 1 
settingsProcessingData(pr1,b1,trial_time,h_freq,l_freq,fs, subject, session,task,repo,path)

# Participant - block 2 
settingsProcessingData(pr2,b2,trial_time,h_freq,l_freq,fs, subject, session,task,repo,path)

# Participant - block 3 
settingsProcessingData(pr3,b3,trial_time,h_freq,l_freq,fs, subject, session,task,repo,path)

# Participant - block 4 
settingsProcessingData(pr4,b4,trial_time,h_freq,l_freq,fs, subject, session,task,repo,path)

# Participant - block 5 
settingsProcessingData(pr5,b5,trial_time,h_freq,l_freq,fs, subject, session,task,repo,path)

# Participant - block 6 
settingsProcessingData(pr6,b6,trial_time,h_freq,l_freq,fs, subject, session,task,repo,path)

# Participant - block 7 
settingsProcessingData(pr7,b7,trial_time,h_freq,l_freq,fs, subject, session,task,repo,path)